In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
#from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import LinearRegression
import pydot
from IPython.display import Image

In [2]:
data = pd.read_csv("BANKNIFTYFUTALL.csv")
feature_cols_JUL = ['trade', 'event', 'ob', 'bevent','btrade','spreadreturn', 'niftyidevent','niftyidindex']
#data.dropna(subset=['return800'], axis=0)

data.drop([data.index[457897]],inplace=True)

In [3]:
X= data[feature_cols_JUL]
Y = data.return800

##### Applying Decision tree Regressor by splitting the dta into train and test

In [30]:
x_train,x_test,y_train,y_test= train_test_split(X,Y,test_size=0.3,random_state=1)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((320527, 8), (137370, 8), (320527,), (137370,))

In [31]:
dt = DecisionTreeRegressor(max_depth=1,min_samples_split=2)
dt.fit(x_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=1, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [6]:
from sklearn.metrics import r2_score,mean_squared_error
y_pred = dt.predict(x_test)
mse=mean_squared_error(y_test,y_pred)
rmse= np.sqrt(mse)
rmse

4.144577992334352

In [7]:
r2_score(y_test,y_pred)

0.02144272390811064

##### Constructing the tree

In [8]:
from sklearn.externals.six import StringIO
from sklearn.tree import export_graphviz  
dot_data = StringIO()
export_graphviz(dt, out_file='dot_data_JUL1.dot',  
                         feature_names= ['trade', 'event', 'ob', 'bevent','btrade','spreadreturn', 'niftyidevent','niftyidindex'],class_names='return800',  
                         filled=True, rounded=True,  
                         special_characters=True)  

##### Applying Linear Regression by dividing the dataset into two parts

In [9]:
from sklearn.linear_model import LinearRegression
Index_label1 = X[X['ob']<-0.517].index.tolist() 
x_model_1 = X.iloc[Index_label1]
y_model_1=Y.iloc[Index_label1]

Index_label2 = X[X['ob']>=-0.517].index.tolist()
x_model_2=X.iloc[Index_label2]
y_model_2 = Y.iloc[Index_label2]

reg1= LinearRegression()
reg1.fit(x_model_1,y_model_1)
reg2= LinearRegression()
reg2.fit(x_model_2,y_model_2)
reg= LinearRegression()
reg.fit(X,Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

##### Comparing the scores of the model Before and After split
- It can be observed that score of model after splitting has increased drastically

In [10]:
r_sq1 = reg1.score(x_model_1,y_model_1)
print("r_sq1 ",r_sq1)
r_sq2 = reg2.score(x_model_2,y_model_2)
print("r_sq2 ",r_sq2)
r_sq = reg.score(X,Y)
print("r_sq ",r_sq)

predict1 = reg1.predict(x_model_1)
predict2 = reg2.predict(x_model_2)
pred = reg.predict(X)

pred= pd.Series(data=pred)
predict1 = pd.Series(data=predict1)
predict2 = pd.Series(data=predict2)

correlation1 = y_model_1.corr(predict1)
correlation2 = y_model_2.corr(predict2)
corre= Y.corr(pred)
print("correlation_after_split :",correlation1)
print("correlation_after_spit :",correlation2)
print("corr_before_split:",corre)

r_sq1  0.08316174003336785
r_sq2  0.01497921178253747
r_sq  0.0034188210916376027
correlation_after_split : 0.009972128635100783
correlation_after_spit : 0.0005395225444653878
corr_before_split: 0.05847068574625569


##### Calculating the correlation between predicted and actual values manually

In [11]:
l1=[]
l2=[]
for i in range(len(Y)):
    if(Y[i]>0):
        l1.append(1)
    elif(Y[i]<0):
        l1.append(-1)
    else:
        l1.append(0)
        
for i in range(len(pred)):
    if(pred[i]>0):
        l2.append(1)
    elif(pred[i]<0):
        l2.append(-1)
    else:
        l2.append(0)

print(len(l1))
print(len(l2))
pred= pd.Series(data=pred)
pred1 = pd.Series(data=l1)
pred2 = pd.Series(data=l2)
corr1 = y_model_1.corr(pred1)
corr2 = y_model_2.corr(pred2)
co= pred1.corr(pred2)
print(co)

457897
457897
0.06299072220140557


##### Applying DecisionTreeRegressor on the data by changing the value of parameters like max_depth and min_samples_split
- Minimum samples for a node split: Defines the minimum number of samples (or observations) which are required in a node to be considered for splitting.

- Minimum samples for a terminal node (leaf): Defines the minimum samples (or observations) required in a terminal node or leaf.

- Maximum depth of the tree (vertical depth): Used to control over-fitting as higher depth will allow the model to learn relations very specific to a particular sample


In [36]:
dt2 = DecisionTreeRegressor(max_depth=2,min_samples_split=2)
dt2.fit(x_train,y_train)
from sklearn.metrics import r2_score,mean_squared_error
y_pred = dt2.predict(x_test)
mse=mean_squared_error(y_test,y_pred)
rmse= np.sqrt(mse)
print("rmse",rmse)

score_train=dt2.score(x_train,y_train)
print("score for train",score_train)
print("score fro test",dt2.score(x_test,y_test))

rmse 4.07156993702865
score for train 0.05735271793947039
score fro test 0.05619492063668153


In [42]:
from sklearn.externals.six import StringIO
from sklearn.tree import export_graphviz  
dot_data = StringIO()
export_graphviz(dt2, out_file='dot_data_JUL2.dot',  
                         feature_names= ['trade', 'event', 'ob', 'bevent','btrade','spreadreturn', 'niftyidevent','niftyidindex'],class_names='return800',  
                         filled=True, rounded=True,  
                         special_characters=True)  

In [34]:
dt3 = DecisionTreeRegressor(max_depth=3,min_samples_split=2)
dt3.fit(x_train,y_train)
from sklearn.metrics import r2_score,mean_squared_error
y_pred = dt3.predict(x_test)
mse=mean_squared_error(y_test,y_pred)
rmse= np.sqrt(mse)
print("rmse",rmse)

score_train=dt3.score(x_train,y_train)
print("score for train",score_train)
print("score fro test",dt3.score(x_test,y_test))

rmse 4.028621643876865
score for train 0.07725580205217153
score fro test 0.07600105457883388


In [43]:
from sklearn.externals.six import StringIO
from sklearn.tree import export_graphviz  
dot_data = StringIO()
export_graphviz(dt3, out_file='dot_data_JUL3.dot',  
                         feature_names= ['trade', 'event', 'ob', 'bevent','btrade','spreadreturn', 'niftyidevent','niftyidindex'],class_names='return800',  
                         filled=True, rounded=True,  
                         special_characters=True)  

In [38]:
dt4 = DecisionTreeRegressor(max_depth=4,min_samples_split=2)
dt4.fit(x_train,y_train)
from sklearn.metrics import r2_score,mean_squared_error
y_pred = dt4.predict(x_test)
mse=mean_squared_error(y_test,y_pred)
rmse= np.sqrt(mse)
print("rmse",rmse)

score_train=dt4.score(x_train,y_train)
print("score for train",score_train)
print("score fro test",dt4.score(x_test,y_test))

rmse 3.903212030335391
score for train 0.13416031200399758
score fro test 0.13263319034912657


In [44]:
from sklearn.externals.six import StringIO
from sklearn.tree import export_graphviz  
dot_data = StringIO()
export_graphviz(dt4, out_file='dot_data_JUL4.dot',  
                         feature_names= ['trade', 'event', 'ob', 'bevent','btrade','spreadreturn', 'niftyidevent','niftyidindex'],class_names='return800',  
                         filled=True, rounded=True,  
                         special_characters=True)  

##### Applying Decision Tree Regression for our dataset
- Decision trees regression normally use mean squared error (MSE) to decide to split a node in two or more sub-nodes
- Here, many different values of test split and train split are taken to check if the model is overfitting or not
- Overfitting occurs when on changing the size of dataset and it parameters the score of train set increases whereas the score of test set decreases.

In [26]:
x_train,x_test,y_train,y_test= train_test_split(X,Y,test_size=0.4,random_state=1)

dt = DecisionTreeRegressor(max_depth=4,min_samples_split=2)
dt.fit(x_train,y_train)

from sklearn.metrics import r2_score,mean_squared_error
y_pred = dt.predict(x_test)
mse=mean_squared_error(y_test,y_pred)
rmse= np.sqrt(mse)
print("rmse",rmse)

score_train=dt.score(x_train,y_train)
print("score for train",score_train)
print("score fro test",dt.score(x_test,y_test))

rmse 3.9044328292587847
score for train 0.13488223352023831
score fro test 0.1315565801961598


##### Below are some reults with different parameters
##for depth=1, split at 0.2,0.3,0.4,0.5
- rmse 4.145847605849474
- score for train 0.02226440625077197
- score fro test 0.02283569743557245
- rmse 4.144504581414698
- score for train 0.022507148317065795
- score fro test 0.022079027012245178
- rmse 4.144577992334352
- score for train 0.022997720054378945
- score fro test 0.021442723908110643
- rmse 4.148930982547038
- score for train 0.02281081351563241
- score fro test 0.021948244176629328

##for depth=2, split at 0.2,0.3,0.4,0.5
- rmse 4.073192484629125
- score for train 0.057076295282211816
- score fro test 0.05678479664169378
- rmse 4.07156993702865
- score for train 0.05735271793947039
- score fro test 0.05619492063668153
- rmse 4.071692793025529
- score for train 0.05789566073372354
- score fro test 0.05555727862956494
- rmse 4.074746247753332
- score for train 0.057165238729801726
- score fro test 0.056611553379002255

##for depth=3, split at 0.2,0.3,0.4
- rmse 4.030468204679393
- score for train 0.07710190366342295
- score fro test 0.07646805192755335
- rmse 4.028621643876865
- score for train 0.07725580205217175
- score fro test 0.0760010545788338
- rmse 4.029022780150299
- score for train 0.07806967694044176
- score fro test 0.07524846012146214

##for depth=4, split at 0.2,0.3,0.4
- rmse 3.906473644615273
- score for train 0.1339057510519709
- score fro test 0.13241762220709163
- rmse 3.903212030335391
- score for train 0.13416031200399758
- score fro test 0.13263319034912668
- rmse 3.9044328292587847
- score for train 0.13488223352023831
- score fro test 0.1315565801961598



In [45]:
regr = BaggingRegressor(base_estimator=LinearRegression(),n_estimators=10, random_state=0).fit(X, Y)
regr.score(X,Y)

0.003417216180843341